In [12]:
import nbformat
from nbconvert import PythonExporter
import os

# Paths
notebook_filename = os.path.expanduser('~/Desktop/ISP Project/Project_code.ipynb')
output_script_filename = os.path.expanduser('~/Desktop/ISP Project/Project_code_modified.py')

# Check if the notebook file exists
if not os.path.isfile(notebook_filename):
    raise FileNotFoundError(f"The notebook file was not found: {notebook_filename}")

# Load the notebook
with open(notebook_filename, 'r', encoding='utf-8') as f:
    notebook_content = nbformat.read(f, as_version=4)

# Convert the notebook to a Python script
python_exporter = PythonExporter()
python_script, _ = python_exporter.from_notebook_node(notebook_content)

# Modify the script to save plots and correct indentation
lines = python_script.split('\n')
lines.insert(0, 'import matplotlib.pyplot as plt\n')

# Function to correct indentation
def correct_indentation(lines):
    corrected_lines = []
    indent_level = 0
    indent_stack = []

    for line in lines:
        stripped_line = line.lstrip()

        if not stripped_line:
            corrected_lines.append('')
            continue

        if stripped_line.startswith(('def ', 'class ', 'for ', 'while ', 'if ', 'elif ', 'else:', 'try:', 'except ', 'with ')):
            indent_stack.append(indent_level)

        if stripped_line.startswith(('return', 'break', 'continue', 'pass', 'raise', 'yield')):
            indent_level = indent_stack.pop() if indent_stack else indent_level - 1

        corrected_line = ' ' * 4 * indent_level + stripped_line

        if stripped_line.endswith(':') and not stripped_line.startswith(('return', 'break', 'continue', 'pass', 'raise', 'yield')):
            indent_level += 1

        corrected_lines.append(corrected_line)

        if stripped_line.startswith(('return', 'break', 'continue', 'pass', 'raise', 'yield')):
            indent_level = indent_stack.pop() if indent_stack else indent_level - 1

    return corrected_lines

# Correct the indentation in the script
corrected_lines = correct_indentation(lines)

# Add a line to save figures after each plot
plot_count = 1
for i, line in enumerate(corrected_lines):
    if 'plt.show()' in line:
        corrected_lines[i] = f'plt.savefig("plot_{plot_count}.png")\nplt.show()\n'
        plot_count += 1

# Save the modified script
with open(output_script_filename, 'w', encoding='utf-8') as f:
    f.write('\n'.join(corrected_lines))

print(f"Modified script saved to {output_script_filename}")


Modified script saved to /Users/christopherspence/Desktop/ISP Project/Project_code_modified.py
